In [1]:
'''
This script extracts values from an Excel file, and outputs it into a json or csv format as specified by the user.
The file names, which sheet to read, what are the ranges of cells, etc are obtained from a user-specified json file.

The json file format is as follows:
{
   "Group1":{
        "fileLoc": "excelfile.xlsx",
        "sheetCell": {
            "SheetName1": [{"header_name": cell_ranges}]
            "SheeetName2": [cell_ranges]
        },
        "jsonOutput": "json_output_path",
        "csvOutput": "csv_output_path"
    },
   "Group2":{ 
            ...
    },
    ... 
}

Where:
Group is to separate between different files and output locations. Each group must at least specify fileLoc
   and sheetCell. 
fileLoc specifies the location of the Excel file whose contents is to be extracted
sheetCell is a dictionary with the sheet names in fileLoc as keys, where the values are either:
 1. Cell ranges in string or list, such as "A4:B6", ["E:G", "B23", "F5:G7"], or
 2. Another dictionary where the keys are a user-defined header names while the values are as specified in 1.
 A few notes:
   - In the case where the header - value in the Excel sheet is horizontally aligned, add "h" in the cell ranges,
        such as "G1:F15h".
   - If no header is specified, the header is assumed to be the first non-empty cell for each row / column,
        depending on the alignments.
   - If a single cell is given, the header is returned as that cell location. 
jsonOutput specifies the output path for json files. If no value is given, there's no output.
csvOutput specifies the output path for csv files. If no value is given, there's no output.
'''

import pandas as pd
import openpyxl
import json
import csv
import win32com.client as win32
from datetime import datetime

class ExcelExport():
    # Initialize 
    def __init__(self, jsonCmd, lts = [], ltc = [], csvOutput = None, jsonOutput = None):
        self.fnout = {}
        self.ltc = []
        self.csvPath = csvOutput
        self.jsonPath = jsonOutput
        with open(jsonCmd) as f:
            self.jcmd = json.load(f)
    
    # Placeholder
    def clear_result(self):
        self.fnout = {}
    
    # Change json file that specifies inputs and outputs
    def chg_file(self, jsonCmd):
        with open(jsonCmd) as f:
            self.jcmd = json.load(f)
    
    # Safety check for multiple inputs with the same key.
    def append_multkey(self, groups):
        plc = {}
        counter = 0
        for element in self.fnout[groups]:
            for key, vals in element.items():
                if key in plc.keys():
                    counter += 1
                    plc.update({key + '_' + str(counter): vals})
                else:
                    plc.update({key: vals})    
        return plc
    
    # Generate tuple for horizontally aligned Excel table
    def gen_table1(self, acSheet, to_extract):
        if any(str.isdigit(char) for char in to_extract):
            return acSheet[to_extract]
        else:
            return tuple(zip(*acSheet[to_extract]))
    
    # Generate tuple for vertically aligned Excel table
    def gen_table2(self, acSheet, to_extract):
        if any(str.isdigit(char) for char in to_extract):
            return tuple(zip(*acSheet[to_extract]))
        else:
            return acSheet[to_extract]

    # For json output
    def jsonOut(self, groups, update = False):
        if self.output.jsonPath != None:
            placeholder = self.append_multkey(groups)
            
            # Remove all None values (empty cells in Excel sheet)
            placeholder = {key: list(filter(None.__ne__, 
                                            vals if isinstance(vals, list) else [vals])) for key, vals in placeholder.items()}
            # Changing list of len 1 to their respective type
            for k in placeholder.keys():
                if len(placeholder[k]) == 1:
                    placeholder[k] = placeholder[k][0]
            placeholder = {groups: placeholder}
            
            fname = self.output.jsonPath.replace('.json', '') + str(datetime.now().date()) + '.json'
            with open(fname, 'w') as f:
                f.write(json.dumps(placeholder, indent = 4))
            print('Data with key', groups, 'successfully exported to', fname, '.')
        else:
            print("No json output file specified for", groups, ".")

    # For csv output
    def csvOut(self, groups):
        if self.output.csvPath != None:
            placeholder = self.append_multkey(groups)
            
            # Remove all None values (empty cells in Excel sheet)
            placeholder = {key: list(filter(None.__ne__, 
                                            vals if isinstance(vals, list) else [vals])) for key, vals in placeholder.items()}
            fname = self.output.csvPath.replace('.csv', '') + str(datetime.now().date()) + '.csv'
            pd.DataFrame(dict([(k, pd.Series(v)) for k, v in placeholder.items()])).to_csv(fname, index = False)
            print('Data with key', groups, 'successfully exported to', fname, '.')
        else:
            print("No csv output file specified for", groups, ".")
    
    # For extracting when the user doesn't define any key / header. 
    def nh_extract(self, sheetname, to_extract):
        acSheet = self.file.content[sheetname]
        
        # Check whether the Excel table is horizontally or vertically aligned, then fixes the openpyxl cell ordering
        if(to_extract[-1] == 'h'):
            to_extract = to_extract[:-1]
            if isinstance(acSheet[to_extract], tuple):
                tbl = self.gen_table1(acSheet, to_extract)
                
                if isinstance(tbl[0], tuple):
                    for elm in tbl:
                        self.ltc.append(headDict(elm, counter = True))
                else:
                    self.ltc.append(headDict(tbl, counter = True))
            
            else:
                self.ltc.append({to_extract: acSheet[to_extract].value})
                # xy = openpyxl.utils.cell.coordinate_from_string(clRg)
                # while(xy[1] != 1 and acSheet[xy[0] + str(xy[1])].value != None): xy = tuple([xy[0], xy[1]-1])
                # self.ltc.append({acSheet[xy[0] + str(xy[1]+1)].value: acSheet[clRg].value})
        
        # Vertically aligned
        else:
            if isinstance(acSheet[to_extract], tuple):
                tbl = self.gen_table2(acSheet, to_extract)
                
                # Appending cell values as key (first cell with value) and values (the rest of the cell in that row / column).
                # Note that 
                if isinstance(tbl[0], tuple):
                    for elm in tbl:
                        self.ltc.append(headDict(elm, counter = True))
                else:
                    self.ltc.append(headDict(tbl, counter = True))

            else:
                self.ltc.append({to_extract: acSheet[to_extract].value})
                # xy = openpyxl.utils.cell.coordinate_from_string(clRg)
                # while(xy[1] != 1 and acSheet[xy[0] + str(xy[1])].value != None): xy = tuple([xy[0], xy[1]-1])
                # self.ltc.append({acSheet[xy[0] + str(xy[1]+1)].value: acSheet[clRg].value})

    # For extracting when the user defines a key / header.
    def wh_extract(self, sheetname, to_extract):
        acSheet = self.file.content[sheetname]
        for hd, cells in to_extract.items():
            if isinstance(cells, str):
                cells = [cells]
            val_holder = []
            
            for clRg in cells:
                # Check whether the Excel table is horizontally or vertically aligned, then fixes the openpyxl cell ordering
                if(clRg[-1] == 'h'):
                    clRg = clRg[:-1]
                    if isinstance(acSheet[clRg], tuple):
                        tbl = self.gen_table1(acSheet, clRg)

                        if isinstance(tbl, tuple):
                            val_holder.extend([elm.value for tupl in tbl for elm in tupl])
                        else:
                            val_holder.append(tbl.value)
                    else:
                        val_holder.append(acSheet[clRg].value)
                # Vertically aligned
                else:
                    if isinstance(acSheet[clRg], tuple):
                        tbl = self.gen_table2(acSheet, clRg)

                        if isinstance(tbl, tuple):
                            val_holder.extend([elm.value for tupl in tbl for elm in tupl])
                        else:
                            val_holder.append(tbl.value)
                    else:
                        val_holder.append(acSheet[clRg].value)
            self.ltc.append({hd: val_holder})
    
    # Performs the whole algorithm for a specified group.
    def automate(self, group):
        self.ltc = []
        self.fnout = {}
        self.formats = self.jcmd[group]
        self.file = files(self.formats['fileLoc'])
        self.sheetdicts = self.formats['sheetCell']
        self.output = output(self.formats['jsonOutput'], self.formats['csvOutput'])

        for sheetname in list(self.sheetdicts.keys()):
            extlist = [self.sheetdicts[sheetname]] if isinstance(
                self.sheetdicts[sheetname], str) else self.sheetdicts[sheetname]
            
            for to_extract in extlist:
                if isinstance(extlist, str):
                    to_extract = extlist
                    
                # Check format of sheetdicts.
                if isinstance(to_extract, str):
                    self.nh_extract(sheetname, to_extract)

                elif isinstance(to_extract, dict):
                    # Case dictionary in the format {headername: vals_to_find}
                    self.wh_extract(sheetname, to_extract)

                else:
                    raise Exception('''Invalid json format. Sheet, cell and header (optional) 
                                    location format should be either \"{\'Sheetname\': \'CellList\'}\" or 
                                    "{\'Sheetname\': {\'Headername\': \'CellList\'}}"''')
                    break

            self.fnout.update({group: self.ltc})

        self.csvOut(group)
        self.jsonOut(group)

    # Performs the algorithm for all groups specified in the json command file.
    def automate_all(self):
        for groups in self.jcmd:
            self.automate(groups)

# For opening the Excel file
class files:
    def __init__(self, fileName):
        self.name = fileName
        if self.name[-3:] == 'xls':
            excel = win32.gencache.EnsureDispatch('Excel.Application')
            wb = excel.Workbooks.Open(self.name)

            wb.SaveAs(self.name+"x", FileFormat = 51)    #FileFormat = 51 is for .xlsx extension
            wb.Close()                                   #FileFormat = 56 is for .xls extension
            excel.Application.Quit()
            
        self.content = openpyxl.load_workbook(fileName, data_only = True)
    
# Specifying the output path.
class output:
    def __init__(self, jsonFile, csvFile):
        self.jsonPath = jsonFile
        self.csvPath = csvFile

    def csv(self, csvFile):
        self.csv = csvFile
    
    def json(self, jsonFile):
        self.json = jsonFile

# Placeholder. If no header is specified and only a single cell value is given, this function returns the header.
# For verticaly aligned Excel table only.
def headDict(tbl, counter = True):
    plc = []
    header = []
    for elm in tbl:
        if elm.value != None:
            if counter:
                header = elm.value
                counter = False
            else:
                plc.append(elm.value)
    if header != []:
        return {header: plc}